In [ ]:
import pandas as pd

features = pd.read_csv("final_df.csv", index_col = 0)
features.head()

,movie,release date,rating,genre,language,directors,duration,Box Office,Description,Audience Score,Tomatometer,Reviews,compound_audience,neutral_audience,positive_audience,negative_audience,positive,negative,neutral,compound
0,chicken_run,"Jun 21, 2000",G,"['animation', 'kids and family', 'comedy', 'ad...",English,['Peter Lord'],1h 25m,NaN,"This engaging stop-motion, claymation adventur...",65.0,97.0,['While Chicken Run has a great animation styl...,0.395363,0.764375,0.208500,0.027125,0.14000,0.0675,0.79250,0.130605
1,the_taste_of_others,"Feb 26, 2002",R,['comedy'],French (France),['Agnès Jaoui'],1h 52m,$635.3K,Castella (Jean-Pierre Bacri) is a successful b...,82.0,100.0,['A nice comedy that has pokes some friendly b...,0.544600,0.700125,0.237250,0.062625,0.25505,0.0301,0.71490,0.400550
2,yi_yi,"May 8, 2001",NaN,['drama'],Edward Yang,['Edward Yang'],2h 53m,$969.1K,"Set in Taiwan, the film follows the lives of t...",91.0,97.0,['A city symphony in minor key with a city-siz...,0.559800,0.806111,0.152556,0.041444,0.20600,0.0403,0.75370,0.348930
3,memento,"Mar 1, 2017",R (Language|Drug Content|Violence),['mystery and thriller'],English,['Christopher Nolan'],1h 56m,$25.5M,Leonard (Guy Pearce) is tracking down the man ...,94.0,93.0,"['Not a masterpiece but really good', ""Memento...",0.326714,0.740143,0.205429,0.054571,0.10860,0.0562,0.83515,0.157130
4,the_life_and_times_of_hank_greenberg,"Jul 2, 1999",PG (Mild Language|Thematic Elements),"['biography', 'documentary']",English,['Aviva Kempner'],1h 29m,$1.7M,"""The Life and Times of Hank Greenberg"" is a hu...",80.0,97.0,"[""Dedicated to the millions of Detroit Tigers ...",0.840233,0.697333,0.289333,0.013333,0.22375,0.0243,0.75195,0.574785


In [ ]:
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

for i in range(len(features['release date'])):
  try:
    features['release date'].iloc[i] = pd.to_datetime(features['release date'].iloc[i])
  except:
    features['release date'].iloc[i] = np.NaN
  

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
features['release date'] = pd.to_datetime(features['release date'])

In [ ]:
features['year'] = features['release date'].dt.year

In [ ]:
features['genre'] = features['genre'].str.split(', ').str[0].str.strip('[]')

In [ ]:
def minute_convertor(x):
  if not pd.isnull(x):
    return int(x[0]) * 60 + int(x[-3:-1])
  return x

In [ ]:
features['duration'] = features['duration'].apply(minute_convertor)

In [ ]:
def box_office_convertor(x):
  if pd.isnull(x):
    return x
  x = x.strip('$')
  
  if x[-1] == 'K':
    return float(x[:-1]) * 1000
  elif x[-1] == 'M':
    return float(x[:-1]) * 1000000
  else:
    return float(x)


In [ ]:
features['rating'] = features['rating'].str.split("(").str[0]

In [ ]:
model = features[['rating', 'genre', 'duration', 'Tomatometer', 'compound_audience', 'neutral_audience', 'positive_audience', 'negative_audience', 'positive', 'negative', 'neutral', 'compound', 'year', 'Audience Score']]

In [ ]:
model_no_nans = model.fillna(model.mean())

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = {
        'max_depth': [3, 5, 7, 10, 13, 15, 18, None],
        'min_samples_split': [2, 3, 5, 7, 10, 15, 20],
        'min_samples_leaf': [2, 3, 5, 7, 10, 15, 20],
        'n_estimators': [400, 800, 1000, 1400, 1800, 2000]
}
clf = GridSearchCV(estimator = RandomForestRegressor(), param_grid = parameters, cv=3)
clf.fit(model_no_nans.drop(['Audience Score', 'rating', 'genre'], axis = 1), model_no_nans[['Audience Score']])


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 

In [ ]:
clf.best_params_

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

In [ ]:
df_train, df_test = train_test_split(model_no_nans, test_size=0.33)

In [ ]:
rf_test = RandomForestRegressor()

In [ ]:
rf_test.fit(df_train.drop(['Audience Score', 'rating', 'genre'], axis = 1), df_train['Audience Score'])

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
preds = rf_test.predict(df_test.drop(['Audience Score', 'rating', 'genre'], axis = 1))

In [ ]:
errors = abs(preds - df_test['Audience Score'])
mape = 100 * (errors / df_test['Audience Score'])

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 87.57 %.


In [ ]:
num_columns = ['duration', 'Tomatometer', 'compound_audience', 'neutral_audience', 'positive_audience', 'negative_audience', 'positive', 'negative', 'neutral', 'compound', 'year']
pl_num = Pipeline([('scaler', StandardScaler())])
preproc = ColumnTransformer(transformers = [('scaler', pl_num, num_columns)], remainder = 'passthrough')
pl_rf = Pipeline(steps=[('preprocessor', preproc), ('rf', RandomForestRegressor())])

In [ ]:
pl_rf.fit(df_train.drop(['Audience Score', 'rating', 'genre'], axis=1), df_train['Audience Score'])

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('scaler',
                                                  Pipeline(memory=None,
                                                           steps=[('scaler',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True))],
                                                           verbose=False),
                                                  ['duration', 'Tomatometer',
                                                   'compound_audience',
                             

In [ ]:
preds = pl_rf.predict(df_test.drop(['Audience Score', 'rating', 'genre'], axis =1))
errors = abs(preds - df_test['Audience Score'])

In [ ]:
mape = 100 * (errors / df_test['Audience Score'])

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 87.52 %.


In [ ]:
import numpy as np

features = features.fillna(features.mean())
targets = np.array(features["Audience Score"])

features = features.drop(["Audience Score","movie", "release date", "rating", "genre", "language", "directors", "Description", "Reviews", "Box Office"], axis=1)
#features = model.drop('Box Office', axis = 1)
features_list = list(features.columns)
features = np.array(features)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


[65. 82. 91. ... 66. 77. 59.]


In [ ]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, targets, test_size = 0.25,
                                                                           random_state = 42)

In [ ]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (1572, 11)
Training Labels Shape: (1572,)
Testing Features Shape: (525, 11)
Testing Labels Shape: (525,)


In [ ]:
# The baseline predictions are the historical averages
baseline_preds = test_features[:, features_list.index('Tomatometer')]

# Baseline errors, and display average baseline error
baseline_errors = abs(baseline_preds - test_labels)
print('Average baseline error: ', round(np.mean(baseline_errors), 2), '%')

Average baseline error:  12.76 %


In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Instantiate model 
rf = RandomForestRegressor(n_estimators= 1000, random_state=42)

# Train the model on training data
rf.fit(train_features, train_labels);

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

# Calculate the absolute errors
errors = abs(predictions - test_labels)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), '%')

Mean Absolute Error: 8.26 %


In [ ]:

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 88.41 %.


In [ ]:
rf.score(test_features, test_labels)

0.10279581072603028